Authors: A.Radhakrishnan, GFDL
Ack: Anderson Banihirwe for intake-esm updates, GFDL colleagues for data

In [90]:
import xarray as xr
import intake
import intake_esm
import sys
from dask_gateway import Gateway
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.options.display.max_rows = None

In [11]:
def launchDask(options):
    #TODO try, except
    gateway = Gateway()
    clusters = gateway.list_clusters()
    clusters
    if len(clusters) >0:
        cluster = gateway.connect(clusters[0].name)
        print("using existing connection")#or shut down and start afresh cluster.shutdown(). otherwise I dunno how to apply custom options
        cluster.shutdown()
        cluster = gateway.new_cluster(options)
    else:
        cluster = gateway.new_cluster(options)
    return(cluster)

In [12]:
gateway = Gateway()
options = gateway.cluster_options()
options.worker_memory=4
cluster = launchDask(options)


In [14]:
min_w = 2
max_w = 60
from distributed import Client
client = Client(cluster)
client

Client Scheduler: gateway://traefik-daskhub-dask-gateway.daskhub:80/daskhub.358b115d4ce04d3280273e4eff5d8e69 Dashboard: /services/dask-gateway/clusters/daskhub.358b115d4ce04d3280273e4eff5d8e69/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [16]:
cluster.adapt(minimum=min_w, maximum=max_w)

esgf-world.json is the ESM collections spec file for the netCDF data in the S3 bucket esgf-world. 
The catalog is updated on an on-demand basis for now. 
You can refer to https://github.com/aradhakrishnanGFDL/gfdl-aws-analysis/tree/community/esm-collection-spec-examples for the most recent catalogs
More examples can be found in https://github.com/aradhakrishnanGFDL/gfdl-aws-analysis/tree/community/examples 


In [17]:
col_url = "https://cmip6-nc.s3.us-east-2.amazonaws.com/esgf-world.json" 

In [18]:
col = intake.open_esm_datastore(col_url)

In [20]:
def latest_version(cat):
    """
    input
    cat: esmdatastore
    output
    esmdatastore with latest DRS versions
    """
    latest_cat = cat.df.sort_values(by=['version','path']).drop_duplicates(['temporal subset','model','mip_table',
                                               'institute','variable','ensemble_member',
                                               'grid_label','experiment_id'],keep='last')
    return latest_cat

In [144]:
query = dict(experiment_id=['1pctCO2','abrupt-4xCO2','historical'],
                 mip_table=['Omon','Ofx'],
                 ensemble_member=["r1i1p1f1","r1i1p1f2"],
                 model=['CESM2','CNRM-CM6-1','UKESM1-0-LL','GFDL-ESM4'],
                 grid_label=['gn'],
                 variable=["zos","tauuo","tauvo","deptho"])

cat_subset = col.search(**query)
cat_latest = latest_version(cat_subset)


In [145]:
esmcol_data = col.esmcol_data
cat_latest = intake.open_esm_datastore(cat_latest,esmcol_data=esmcol_data)

In [146]:
cat_latest.df.groupby(['variable','model','experiment_id','version']).nunique()
#the temporal_subset count and path indicate the number of actual files/objects corresponding for each condition

project  institute  frequency  \
variable model       experiment_id version                                    
deptho   CESM2       1pctCO2       v20190425        1          1          0   
                     abrupt-4xCO2  v20190927        1          1          0   
                     historical    v20190308        1          1          0   
         CNRM-CM6-1  1pctCO2       v20180626        1          1          0   
                     abrupt-4xCO2  v20180705        1          1          0   
                     historical    v20180917        1          1          0   
         GFDL-ESM4   1pctCO2       v20180701        1          1          0   
                     abrupt-4xCO2  v20180701        1          1          0   
                     historical    v20190726        1          1          0   
tauuo    CESM2       1pctCO2       v20190425        1          1          1   
                     abrupt-4xCO2  v20190927        1          1          1   
                     historical    v20190308        1          1          1   
         CNRM-CM6-1  1pctCO2       v20180626        1          1          1   
                     abrupt-4xCO2  v20180705        1          1          1   
                     historical    v20180917        1          1          1   
         GFDL-ESM4   1pctCO2       v20180701        1          1          1   
                     abrupt-4xCO2  v20180701        1          1          1   
                     historical    v20190726        1          1          1   
         UKESM1-0-LL 1pctCO2       v20190701        1          1          1   
                     abrupt-4xCO2  v20190701        1          1          1   
                     historical    v20190627        1          1          1   
tauvo    CESM2       1pctCO2       v20190425        1          1          1   
                     abrupt-4xCO2  v20190927        1          1          1   
                     historical    v20190308        1          1          1   
         CNRM-CM6-1  1pctCO2       v20180626        1          1          1   
                     abrupt-4xCO2  v20180705        1          1          1   
                     historical    v20180917        1          1          1   
         GFDL-ESM4   1pctCO2       v20180701        1          1          1   
                     abrupt-4xCO2  v20180701        1          1          1   
                     historical    v20190726        1          1          1   
         UKESM1-0-LL 1pctCO2       v20190406        1          1          1   
                     abrupt-4xCO2  v20190406        1          1          1   
                     historical    v20190406        1          1          1   
zos      CESM2       1pctCO2       v20190425        1          1          1   
                     abrupt-4xCO2  v20190927        1          1          1   
                     historical    v20190308        1          1          1   
         CNRM-CM6-1  1pctCO2       v20180626        1          1          1   
                     abrupt-4xCO2  v20180705        1          1          1   
                     historical    v20180917        1          1          1   
         GFDL-ESM4   1pctCO2       v20180701        1          1          1   
                     abrupt-4xCO2  v20180701        1          1          1   
                     historical    v20190726        1          1          1   
         UKESM1-0-LL 1pctCO2       v20190701        1          1          1   
                     abrupt-4xCO2  v20190701        1          1          1   
                     historical    v20190627        1          1          1   

                                              modeling_realm  mip_table  \
variable model       experiment_id version                                
deptho   CESM2       1pctCO2       v20190425               0          1   
                     abrupt-4xCO2  v20190927               0          1   
                     historical 

In [137]:
#print as needed cat_latest.df

CESM2 Temporal subset 010101-015012 and 000101-005012 is now present from zos in the 1pctCO2 

In [147]:
cat_latest.df.loc[(cat_latest.df['model'] =='CESM2') & (cat_latest.df['variable'] =='zos') & (cat_latest.df['experiment_id'] =='1pctCO2')]


,project,institute,model,experiment_id,frequency,modeling_realm,mip_table,ensemble_member,grid_label,variable,temporal subset,version,path
49,CMIP6,NCAR,CESM2,1pctCO2,mon,ocean,Omon,r1i1p1f1,gn,zos,000101-005012,v20190425,s3://esgf-world/CMIP6/CMIP/NCAR/CESM2/1pctCO2/r1i1p1f1/Omon/zos/gn/v20190425/zos_Omon_CESM2_1pctCO2_r1i1p1f1_gn_000101-005012.nc
50,CMIP6,NCAR,CESM2,1pctCO2,mon,ocean,Omon,r1i1p1f1,gn,zos,005101-010012,v20190425,s3://esgf-world/CMIP6/CMIP/NCAR/CESM2/1pctCO2/r1i1p1f1/Omon/zos/gn/v20190425/zos_Omon_CESM2_1pctCO2_r1i1p1f1_gn_005101-010012.nc
51,CMIP6,NCAR,CESM2,1pctCO2,mon,ocean,Omon,r1i1p1f1,gn,zos,010101-015012,v20190425,s3://esgf-world/CMIP6/CMIP/NCAR/CESM2/1pctCO2/r1i1p1f1/Omon/zos/gn/v20190425/zos_Omon_CESM2_1pctCO2_r1i1p1f1_gn_010101-015012.nc


In [153]:
#there is an issue opening the following in intake-esm. Removing from the dataframe for now
cat_latest.df.loc[(cat_latest.df['model'] =='CESM2') & (cat_latest.df['variable'] =='deptho')  & (cat_latest.df['experiment_id'] =='1pctCO2')]

cat_latest.df = cat_latest.df.query("path!='s3://esgf-world/CMIP6/CMIP/NCAR/CESM2/1pctCO2/r1i1p1f1/Ofx/deptho/gn/v20190425/deptho_Ofx_CESM2_1pctCO2_r1i1p1f1_gn.nc'")

In [ ]:
dset_dict = cat_latest.to_dataset_dict(storage_options=dict(anon=True),cdf_kwargs={'decode_times': False, 'chunks': {'time': 1}})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


In [ ]:
ds = dset_dict["CMIP6.CNRM-CERFACS.CNRM-CM6-1.1pctCO2.Omon"]

In [ ]:
ds.zos

In [ ]:
ds.deptho

In [ ]:
cluster.close()